In [22]:
import paramiko
import schedule
import time

In [23]:
# INIT関数（サーバ内データクリア）
def clear_server(HOST,PORT,SFTP_USER,SFTP_PASSWORD,REMOTE_DIR):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    client.connect(HOST,port=PORT,username=SFTP_USER,password=SFTP_PASSWORD)
    
    sftp_connection = client.open_sftp()
    sftp_connection.chdir(REMOTE_DIR)
    print(sftp_connection.listdir())
    
    for _files in sftp_connection.listdir():
        if 'jpg' in _files:
            sftp_connection.remove(REMOTE_DIR+'/'+_files)
    sftp_connection.close()

In [24]:
for param in remote_param:
    clear_server(param['HOST'],param['PORT'],param['USER'],param['PASS'],remote_dir)

['videos']
['LensCorr.map', 'FusionIRImage.fff', 'videos']
['LensCorr.map', 'FusionIRImage.fff', 'videos']
['LensCorr.map', 'FusionIRImage.fff', 'videos']
['LensCorr.map', 'FusionIRImage.fff', 'videos']
['LensCorr.map', 'FusionIRImage.fff', 'videos']


In [25]:
# 接続するカメラの設定（6台分）
remote_param = []
remote_param.append({'HOST':'192.168.32.101','PORT':22,'USER':'fliruser','PASS':'3vlig'})
remote_param.append({'HOST':'192.168.32.102','PORT':22,'USER':'fliruser','PASS':'3vlig'})
remote_param.append({'HOST':'192.168.32.103','PORT':22,'USER':'fliruser','PASS':'3vlig'})
remote_param.append({'HOST':'192.168.32.104','PORT':22,'USER':'fliruser','PASS':'3vlig'})
remote_param.append({'HOST':'192.168.32.105','PORT':22,'USER':'fliruser','PASS':'3vlig'})
remote_param.append({'HOST':'192.168.32.106','PORT':22,'USER':'fliruser','PASS':'3vlig'})

# ダウンロード元と先の設定
remote_dir = '/FLIR/images' # ダウンロード元
download_dir = 'Desktop\\Flir\\'# ダウンロード先の共通フォルダ

In [26]:
def init_connect_sftp(HOST,PORT,SFTP_USER,SFTP_PASSWORD):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    client.connect(HOST,port=PORT,username=SFTP_USER,password=SFTP_PASSWORD)
    
    return client

In [27]:
sftp_client = []
for param in remote_param:
    sftp_client.append(init_connect_sftp(param['HOST'],param['PORT'],param['USER'],param['PASS']))

In [28]:
def sftp_download(HOST,PORT,SFTP_USER,SFTP_PASSWORD,REMOTE_DIR,DOWNLOAD_DIR,client):
    sftp_connection = client.open_sftp()
    sftp_connection.chdir(REMOTE_DIR)
    
    for _files in sftp_connection.listdir():
        if '.jpg' in _files:
            sftp_connection.get(_files,DOWNLOAD_DIR + HOST + '\\' +_files)
            sftp_connection.remove(REMOTE_DIR+'/'+_files)
    sftp_connection.close()

In [29]:
def job():
    for param,client in zip(remote_param,sftp_client):
        sftp_download(param['HOST'],param['PORT'],param['USER'],param['PASS'],remote_dir,download_dir,client)

In [30]:
# 3分ごとに実行
schedule.every(1).minutes.do(job)

Every 1 minute do job() (last run: [never], next run: 2020-07-06 15:43:33)

In [31]:
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 